# Reading an Example Paper

## Initialization

In [9]:
from ..paperparser.parse.spincoat import parse_spincoat

ValueError: attempted relative import beyond top-level package

## Example Paper

In [2]:
# text from example paper-- import from somewhere? or copypaste
# nature materials: "solvent engineering..."

## Output

The paper parsing will proceed through the following steps:

1. Identification of key paragraphs and separating them into key sentences
2. Parsing synthetic parameters from key synthesis sentences
3. Parsing power conversion efficiency (PCE; a performance metric) from key performance paragraph(s)
4. Organization of synthetic parameters and power conversion efficiency data into structured (ordered and nested) format

In [3]:
# feed paper into step 1 function

In [4]:
# feed step 1 (synth) output into step 2

In [5]:
# feed step 1 (pce) output into step 3

In [6]:
# feed steps 2 and 3 outputs into step 4

In [7]:
# print final output